In [1]:
#!/usr/bin/python
import sys
import pickle
sys.path.append("../tools/")

In [2]:
import numpy as np
import pandas as pd
from collections import Counter
import pprint

# ref: https://github.com/Corvids/ud120-projects (includes all module 5 starter code)
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

### Task 1: Load the Dataset and Initial Features

#### Question 1:

**Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those?  [relevant rubric items: “data exploration”, “outlier investigation”]**

The goal of this project is to identify Enron Employees who may have committed fraud based on the available financial and email dataset using machine learning.

A major outlier in the dataset was the "Total" data point, which was removed, as seen in the following code.

In [3]:
### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list = ['poi', 'salary', 'deferral_payments', 'total_payments',
                 'loan_advances', 'bonus', 'restricted_stock_deferred',
                 'deferred_income', 'total_stock_value', 'expenses',
                 'exercised_stock_options', 'other', 'long_term_incentive',
                 'restricted_stock', 'director_fees',
                 'to_messages', 'from_poi_to_this_person',
                 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi']

### Task 2: Understanding the Dataset and Question

In [4]:
### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

if len(data_dict) > 0:
    print 'data loaded!'
    print 'Number of initial data points: ', len(data_dict)
    print 'Number of initial features used: ', len(features_list)

data loaded!
Number of initial data points:  146
Number of initial features used:  20


In [ ]:
# get number of POI in data
poi_count = []
for key in data_dict.keys():
    poi_count.append(data_dict[key]['poi'])

print 'Number of POI: ', poi_count.count(1)
print 'Number of non-POI: ', poi_count.count(0)

In [ ]:
print 'list of all people in the dataset: '
print data_dict.keys()

In [ ]:
# get number of missing values per person
missing_feature = [0 for i in range(0, len(features_list))]
features_in_data = data_dict.values()

for loc_p, person in enumerate(features_in_data):
    for loc_f, feature in enumerate(features_list):
        if person[feature] == 'NaN':
            missing_feature[loc_f] += 1

In [ ]:
print 'Number of missing from each feature: '
for feature, num_missing in zip(features_list, missing_feature):
    print feature, ' -- ', num_missing

In [ ]:
# print structure
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(data_dict["SKILLING JEFFREY K"])

##### Removing the outlier

In [ ]:
import matplotlib.pyplot

%matplotlib inline

features = ["salary", "bonus"]
data = featureFormat(data_dict, features)

In [ ]:
for point in data:
    salary = point[0]
    bonus = point[1]
    matplotlib.pyplot.scatter( salary, bonus )

matplotlib.pyplot.xlabel("salary")
matplotlib.pyplot.ylabel("bonus")
matplotlib.pyplot.show()

In [ ]:
### Task 2: Remove outliers
##following outlier cleaner adapted
## from outliers mini-project

# remove total -- top right point separate from bottom left cluster
print 'Now removing outliers . . . '
print 'Removing point named "Total" from data...'
data_dict.pop('TOTAL')
print 'Number of data points after removal: ', len(data_dict)

In [ ]:
data = featureFormat(data_dict, features)

for point in data:
    salary = point[0]
    bonus = point[1]
    matplotlib.pyplot.scatter( salary, bonus )

matplotlib.pyplot.xlabel("salary")
matplotlib.pyplot.ylabel("bonus")
matplotlib.pyplot.show()

### Task 3: Optimize Feature Selection/Engineering

#### Question 2:

**What features did you end up using in your POI identifier, and what selection process did you use to pick them? Did you have to do any scaling? Why or why not? As part of the assignment, you should attempt to engineer your own feature that does not come ready-made in the dataset -- explain what feature you tried to make, and the rationale behind it. (You do not necessarily have to use it in the final analysis, only engineer and test it.) In your feature selection step, if you used an algorithm like a decision tree, please also give the feature importances of the features that you use, and if you used an automated feature selection function like SelectKBest, please report the feature scores and reasons for your choice of parameter values.  [relevant rubric items: “create new features”, “properly scale features”, “intelligently select feature”]**

TBD

In [ ]:
### Task 3: Create new feature(s)

### Store to my_dataset for easy export below.
my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [ ]:
for key in my_dataset:
    #messages to POI
    if my_dataset[key]['from_messages'] != 'NaN':
        my_dataset[key]['to_poi_message_ratio'] = \
                1.0*my_dataset[key]['from_this_person_to_poi']/my_dataset[key]['from_messages']
    else:
        my_dataset[key]['to_poi_message_ratio'] = 'NaN'
    #messages from POI
    if my_dataset[key]['to_messages'] != 'NaN':
        my_dataset[key]['from_poi_message_ratio'] = \
                1.0*my_dataset[key]['from_poi_to_this_person']/my_dataset[key]['to_messages']
    else:
        my_dataset[key]['from_poi_message_ratio'] = 'NaN'

In [ ]:
# update the feature list
features_list = ['poi', 'salary', 'deferral_payments', 'total_payments',
                 'loan_advances', 'bonus', 'restricted_stock_deferred',
                 'deferred_income', 'total_stock_value', 'expenses',
                 'exercised_stock_options', 'other', 'long_term_incentive',
                 'restricted_stock', 'director_fees',
                 'to_messages', 'from_poi_to_this_person',
                 'from_messages', 'from_this_person_to_poi',
                 'shared_receipt_with_poi',
                 'to_poi_message_ratio', 'from_poi_message_ratio']


In [ ]:
pp.pprint(my_dataset["SKILLING JEFFREY K"])

In [ ]:
if len(data_dict) > 0:
    print 'After adding two new features: to_poi_message_ratio, from_poi_message_ratio'
    print 'Number of initial data points: ', len(my_dataset)
    print 'Number of initial features used: ', len(features_list)

##### Scaling our features

In [ ]:
def scaleFeatures(my_dataset, feature):
    min_scale = np.inf
    max_scale = -np.inf
    
    for key in my_dataset:
        # get min, max
        if my_dataset[key][feature] == 'NaN':
            pass
        else:
            if my_dataset[key][feature] < min_scale:
                min_scale = my_dataset[key][feature]
            if my_dataset[key][feature] > max_scale:
                max_scale =  my_dataset[key][feature]
    
    print 'min ' + str(feature) + ' is: ' + str(min_scale)
    print 'max ' + str(feature) + ' is: ' + str(max_scale)
    
    for key in my_dataset:
        if my_dataset[key][feature] == 'NaN':
            pass
        else:
            my_dataset[key][feature] = 1.0*(my_dataset[key][feature] - min_scale) / (max_scale - min_scale)
    
    print 'scaled feature ' + str(feature) + '!'
    print '=========='

In [ ]:
features_to_scale = ['bonus', 'salary', 'restricted_stock',
                     'long_term_incentive',
                     'restricted_stock_deferred', 'total_payments',
                    'total_stock_value', 'exercised_stock_options',
                    'expenses']

for feature in features_to_scale:
    scaleFeatures(my_dataset, feature)

In [ ]:
pp.pprint(my_dataset["SKILLING JEFFREY K"])

In [ ]:
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

##### Feature Selection

In [ ]:
from sklearn.cross_validation import train_test_split

features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [ ]:
from sklearn import tree
from sklearn.metrics import accuracy_score

clf = tree.DecisionTreeClassifier(random_state = 42)
clf = clf.fit(features_train, labels_train)

features_importance = zip(map(lambda x: round(x, 6), clf.feature_importances_), features_list)
features_importance.sort(key = lambda t: t[0], reverse = True)

print 'feature importances: ', features_importance

In [ ]:
n_most = features_importance[0:22] # all points with feature importance > 0.0

selected_features = [x[1] for x in n_most]
print selected_features

In [ ]:
data = featureFormat(my_dataset, selected_features, sort_keys = True)
labels, features = targetFeatureSplit(data)

### Task 4: Pick and Tune an Algorithm

#### Question 3:

**What algorithm did you end up using? What other one(s) did you try? How did model performance differ between algorithms?  [relevant rubric item: “pick an algorithm”]**

TBD

In [ ]:
from sklearn.cross_validation import train_test_split

features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [ ]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html


# Provided to give you a starting point. Try a variety of classifiers.

# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from time import time

clf_GNB = GaussianNB()

t0_GNB = time()
clf_GNB.fit(features_train, labels_train)
print "training time:", round(time()-t0_GNB, 3), "s"

t1_GNB = time()
pred_GNB = clf_GNB.predict(features_test)
print "predicting time:", round(time()-t1_GNB, 3), "s"

accuracy_GNB = accuracy_score(labels_test, pred_GNB)
print 'accuracy of Naive Bayes: ' + str(accuracy_GNB)

In [ ]:
# Decision Tree

from sklearn import tree

clf_tree = tree.DecisionTreeClassifier(min_samples_split=20)

t0_tree = time()
clf_tree.fit(features_train, labels_train)
print "training time:", round(time()-t0_tree, 3), "s"

t1_tree = time()
pred_tree = clf_tree.predict(features_test)
print "predicting time:", round(time()-t1_tree, 3), "s"

accuracy_tree = accuracy_score(labels_test, pred_tree)
print 'accuracy of Decision Tree: ' + str(accuracy_tree)

In [ ]:
# Ada Boost
from sklearn.ensemble import AdaBoostClassifier

clf_Ada = AdaBoostClassifier(n_estimators=5, learning_rate=0.15, random_state=42)

t0_Ada = time()
clf_Ada.fit(features_train, labels_train)
print "training time:", round(time()-t0_Ada, 3), "s"

t1_Ada = time()
pred_Ada = clf_Ada.predict(features_test)
print "predicting time:", round(time()-t1_Ada, 3), "s"

accuracy_Ada = accuracy_score(labels_test, pred_Ada)
print 'accuracy of Ada Boost: ' + str(accuracy_Ada)

In [ ]:
# SVM
from sklearn import svm

clf_SVM = svm.SVC(random_state = 42)

t0_SVM = time()
clf_SVM.fit(features_train, labels_train)
print "training time:", round(time()-t0_SVM, 3), "s"

t1_SVM = time()
pred_SVM = clf_SVM.predict(features_test)
print "predicting time:", round(time()-t1_SVM, 3), "s"

accuracy_SVM = accuracy_score(labels_test, pred_SVM)
print 'accuracy of SVM: ' + str(accuracy_SVM)

### Task 5

#### Question 4:

**What does it mean to tune the parameters of an algorithm, and what can happen if you don’t do this well?  How did you tune the parameters of your particular algorithm? (Some algorithms do not have parameters that you need to tune -- if this is the case for the one you picked, identify and briefly explain how you would have done it for the model that was not your final choice or a different model that does utilize param**

#### Question 5:

**What is validation, and what’s a classic mistake you can make if you do it wrong? How did you validate your analysis?  [relevant rubric item: “validation strategy”]**

#### Question 6:

**Give at least 2 evaluation metrics and your average performance for each of them.  Explain an interpretation of your metrics that says something human-understandable about your algorithm’s performance. [relevant rubric item: “usage of evaluation metrics”]**

In [ ]:
### Task 5: Tune your classifier to achieve better than .3 precision and recall
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info:
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from tester import test_classifier
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeRegressor
from sklearn import grid_search

cv_sets = StratifiedShuffleSplit(labels, n_iter = 10, test_size = 0.20, random_state = 42)
regressor = DecisionTreeRegressor(random_state = 42)
params = {'max_depth':[1,2,3,4,5,6,7,8,9,10]} 

grid = grid_search.GridSearchCV(regressor, param_grid=params, cv=cv_sets)

#grid = grid.fit(features, labels)

test_classifier(grid, my_dataset, features_list, folds = 1000)

In [ ]:
from sklearn import grid_search
from sklearn.metrics import make_scorer, f1_score
from sklearn import cross_validation


parameters = {'kernel':['linear', 'rbf'], 'C':[1,2,3], 'gamma':[0.0]}

# our chosen classifier
clf = svm.SVC(random_state = 42)

f1_scorer = make_scorer(f1_score, pos_label='yes')
grid_obj = grid_search.GridSearchCV(clf, parameters, scoring=f1_scorer)

grid_obj = grid_obj.fit(features_train, labels_train)
clf = grid_obj.best_estimator_

# final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, features_train, labels_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, features_test, labels_test))

### Task 6

In [ ]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)